In [1]:
import pandas as pd

Reading in training data


In [2]:
df1 = pd.read_csv('finnum/train.csv')
x = df1.index
df2 = pd.read_csv('finnum/dev.csv')
df = pd.concat([df1,df2], ignore_index = True)
df.head(10)

,id,idx,tweet,category,subcategory,target_num
0,98221616,4976,$ARNA APD334 for Amyotrophic Lateral Sclerosis...,Product Number,Product Number,334
1,82321187,9839,"$OCLR Noob investor that i am, put a 7.38 stop...",Monetary,stop loss,7.38
2,103328840,1455,$ES_F $SPY Bias-2 bearish and the DLT-1 DRR ar...,Product Number,Product Number,1
3,104840294,1111,$TMUS its acquisition of Layer3 TV The purchas...,Product Number,Product Number,5
4,69935467,2373,$TWTR ^Buy $WSTL 68c up 14% 4 time avg vol. ...,Percentage,relative,14
5,69935467,2373,$TWTR ^Buy $WSTL 68c up 14% 4 time avg vol. ...,Quantity,Quantity,4
6,69935467,2373,$TWTR ^Buy $WSTL 68c up 14% 4 time avg vol. ...,Monetary,forecast,5
7,94249158,1372,$SEED L2 Capital deal is real savvy. It takes ...,Temporal,date,33
8,100979260,505,$BTE $BTE.CA $MEG.CA $CPG $CPG.CA $CJ.CA - 4th...,Temporal,date,4
9,100775772,1210,$WRN My fav $WRN pattern on my watchlist for 1...,Temporal,date,11


Make new column for encoding categories as numbers

In [3]:
df['cat_num'] = df['category'].astype('category').cat.codes

Preprocessing tweets by lowercasing

In [4]:
df['lower'] = [x.lower() for x in df.tweet]

In [5]:
df['lower'].head()

0    $arna apd334 for amyotrophic lateral sclerosis...
1    $oclr noob investor that i am, put a 7.38 stop...
2    $es_f $spy bias-2 bearish and the dlt-1 drr ar...
3    $tmus its acquisition of layer3 tv the purchas...
4    $twtr ^buy  $wstl 68c up 14%  4 time avg vol. ...
Name: lower, dtype: object

Replacing target with <num\>

In [6]:
import re
def flagNum(x):
    text = x.lower
    outNum = str(x.target_num)
    text_out = re.sub(r'(?<=\D)'+outNum+'(?=\D)', ' <num> ', text)
    #text_out = text.replace('\D('+outNum+')\D', ' <num> ')
    return text_out

Applying to training, making this into a new column

In [7]:
df['mod'] = df.apply(lambda x: flagNum(x), axis = 1)

In [8]:
df['mod'].head()

0    $arna apd <num>  for amyotrophic lateral scler...
1    $oclr noob investor that i am, put a  <num>  s...
2    $es_f $spy bias-2 bearish and the dlt- <num>  ...
3    $tmus its acquisition of layer3 tv the purchas...
4    $twtr ^buy  $wstl 68c up  <num> %  4 time avg ...
Name: mod, dtype: object

Filtering out numbers and words less than 3 characters long

In [9]:
import string
def textPuncandNum(text):
    table = str.maketrans('', '', string.punctuation)
    text = re.sub(r'[0-9]+', '', text)
    text = text.split()
    text = [word for word in text if len(word.translate(table))>2]
    return ' '.join(text)
stripped = [textPuncandNum(text) for text in df['mod']]

In [10]:
split = [x.split() for x in stripped]

In [11]:
def replace_contraction(text):
    contraction_patterns = [ (r'won\'t', 'will not'), (r'can\'t', 'can not'), (r'i\'m', 'i am'), (r'ain\'t', 'is not'), (r'(\w+)\'ll', '\g<1> will'), (r'(\w+)n\'t', '\g<1> not'),
                         (r'(\w+)\'ve', '\g<1> have'), (r'(\w+)\'s', '\g<1> is'), (r'(\w+)\'re', '\g<1> are'), (r'(\w+)\'d', '\g<1> would'), (r'&', 'and'), (r'dammit', 'damn it'), (r'dont', 'do not'), (r'wont', 'will not') ]
    patterns = [(re.compile(regex), repl) for (regex, repl) in contraction_patterns]
    for (pattern, repl) in patterns:
        (text, count) = re.subn(pattern, repl, text)
    return text
def replace_links(text, filler=' '):
        text = re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*',
                      filler, text).strip()
        return text
def remove_numbers(text):
    text = ''.join([i for i in text if not i.isdigit()])
    return text

In [12]:
import re
import numpy as np
def cleanText(text):
    text = text.strip().replace("\n", " ").replace("\r", " ")
    text = replace_contraction(text)
    text = replace_links(text, "link")
    text = remove_numbers(text)
    text = re.sub(r'[,!%^&*)(|";:.?\'\\}{]',"",text)
    text = text.lower()
    return text

In [13]:
words = df["mod"].apply(cleanText)
max_length = max(len(lst.split()) for lst in words)
tokens_length = np.asarray([len(x.split()) for x in words])
t_pad = np.asarray([lst.split() + [''] * (max_length - len(lst.split())) for lst in words])

## Implementing Convolutional Layer

In [14]:
import tensorflow as tf
import tensorflow_hub as hub
batch_size = 64

W0418 12:11:24.174127  1708 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [15]:
tf.reset_default_graph()

In [16]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
def ELMoEmbedding(x,y):
    return elmo(inputs = {"tokens": tf.squeeze(tf.cast(x, tf.string)), "sequence_len":tf.cast(y, tf.int32)}, signature="tokens", as_dict=True)["elmo"]

Instructions for updating:
Colocations handled automatically by placer.


W0418 12:11:25.503266  1708 deprecation.py:323] From C:\Users\justi\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\ops\control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Creating the embedding matrix for any input


In [17]:
trainInp = t_pad[x, :]
valInp = t_pad[len(x):, :]

t_len = tokens_length[x]
v_len = tokens_length[len(x):]

labels = pd.get_dummies(df['cat_num'][x])
labels2 = pd.get_dummies(df['cat_num'][len(x):])

Batching and creating iterators

In [18]:
train = (trainInp, t_len, labels)
val = (valInp, v_len, labels2)

# create training Dataset and batch it
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000) # if you want to shuffle your data
train_data = train_data.batch(batch_size)

# create validation Dataset and batch it
val_data = tf.data.Dataset.from_tensor_slices(val)
val_data = val_data.shuffle(10000) # if you want to shuffle your data
val_data = val_data.batch(batch_size)

# create one iterator and initialize it with different datasets
iterator = tf.data.Iterator.from_structure(train_data.output_types, 
                                           train_data.output_shapes)
txt, length, label = iterator.get_next()

train_init = iterator.make_initializer(train_data)
val_init = iterator.make_initializer(val_data)

In [19]:
elmos = ELMoEmbedding(txt, length)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0418 12:11:27.929896  1708 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [20]:
n_neurons = 500
n_neurons2 = 100
n_outputs = 7

In [21]:
basic_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=n_neurons)
cell = tf.nn.rnn_cell.DropoutWrapper(basic_cell, output_keep_prob=0.6)
outputs, states = tf.nn.dynamic_rnn(cell, elmos, dtype=tf.float32)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


W0418 12:11:28.034616  1708 deprecation.py:323] From <ipython-input-21-5bab1b968b7f>:1: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


W0418 12:11:28.044214  1708 deprecation.py:323] From <ipython-input-21-5bab1b968b7f>:3: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


W0418 12:11:28.181537  1708 deprecation.py:506] From C:\Users\justi\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\ops\rnn_cell_impl.py:1259: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [22]:
states[0]

<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 500) dtype=float32>

In [23]:
learning_rate = 0.001
dense = tf.layers.dense(states[0], n_neurons2)
dropout = tf.layers.dropout(dense, 0.5)
logits = tf.layers.dense(dropout, n_outputs)
xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=label,logits=logits)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

Instructions for updating:
Use keras.layers.dense instead.


W0418 12:11:28.253345  1708 deprecation.py:323] From <ipython-input-23-a82746380ddf>:2: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


Instructions for updating:
Use keras.layers.dropout instead.


W0418 12:11:28.287255  1708 deprecation.py:323] From <ipython-input-23-a82746380ddf>:3: dropout (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dropout instead.


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



W0418 12:11:28.312187  1708 deprecation.py:323] From <ipython-input-23-a82746380ddf>:5: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [ ]:
logits = tf.cast(logits, tf.float32)
label = tf.cast(label, tf.int32)

In [ ]:
#correct = tf.nn.in_top_k(logits, label, 1)
correct = tf.equal(tf.argmax(logits, 1), tf.argmax(label, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float64))

In [ ]:
n_epochs = 10

with tf.Session() as sess:
    #start_time = time.time()
    sess.run(tf.global_variables_initializer())
    # train the model n_epochs times

    for i in range(n_epochs): 
        
        sess.run(train_init)# drawing samples from train_data
        total_loss = 0
        total_right = 0
        n_batches = 0
        totalright = 0
        totalvalright = 0
        nvalbatches = 0
        totalvalloss = 0
        try:
            while True:
                #summary,acc,_, l = sess.run([summary_op,accuracy,optimizer, loss]) #use with scalar summary
                acc,_, l = sess.run([accuracy, training_op, loss])                
                total_loss += l
                total_right += acc
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        
        sess.run(val_init)
        try:
            while True:
                accval,valloss = sess.run([accuracy, loss])
                totalvalright += accval
                nvalbatches += 1
                totalvalloss += valloss
        except tf.errors.OutOfRangeError:
            pass

        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
        print('Accuracy {0}: {1}'.format(i, total_right/n_batches))    
        print('Average val loss epoch {0}: {1}'.format(i, totalvalloss/nvalbatches))
        print('Val_Accuracy {0}: {1}'.format(i, totalvalright/nvalbatches)) 
    #prediction = sess.run(preds, feed_dict={txt: valInp})
    #prediction = np.asarray(prediction)
   

Average loss epoch 0: 1.368954762958345
Accuracy 0: 0.5019940476190476
Average val loss epoch 0: 1.2068225145339966
Val_Accuracy 0: 0.5723958333333333
Average loss epoch 1: 0.9180943307422457
Accuracy 1: 0.677625
Average val loss epoch 1: 0.790621226032575
Val_Accuracy 1: 0.7205729166666667
Average loss epoch 2: 0.5465610231672015
Accuracy 2: 0.810952380952381
Average val loss epoch 2: 0.5973976105451584
Val_Accuracy 2: 0.7997395833333334
Average loss epoch 3: 0.32970965391113644
Accuracy 3: 0.8889047619047619
Average val loss epoch 3: 0.6637147963047028
Val_Accuracy 3: 0.8143229166666667
Average loss epoch 4: 0.2076866898508299
Accuracy 4: 0.9336130952380952
Average val loss epoch 4: 0.7145894144972166
Val_Accuracy 4: 0.8151041666666666
Average loss epoch 5: 0.1336665708039488
Accuracy 5: 0.9569107142857143
Average val loss epoch 5: 0.7508332033952078
Val_Accuracy 5: 0.8080729166666667
Average loss epoch 6: 0.08701327966437453
Accuracy 6: 0.9743630952380953
Average val loss epoch 6: 0